In [ ]:
from IPython import display
display.HTML('''<script>
code_show=true;
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
}
$( document ).ready(code_toggle);
</script>
<a href="javascript:code_toggle()">Toggle raw code</a>.''')

In [ ]:
import collections

from IPython import display
from ipywidgets import widgets

out = collections.defaultdict(widgets.Output)
plots_out = collections.defaultdict(
    lambda: widgets.Output(layout={'height': '300px', 'width': '600px', 'overflow': 'scroll'}))

In [ ]:
display.display(out['plots_UI'])

In [ ]:
display.display(plots_out['0'])

In [ ]:
display.display(plots_out['1'])

In [ ]:
display.display(plots_out['2'])

In [ ]:
display.display(plots_out['3'])

In [ ]:
display.display(out['submissions_viewer'])

In [ ]:
from submissions_viewer.tests_results.controller import Controller

controller = Controller(
    ui_out=out['plots_UI'],
    plots_out=[
        plots_out['0'],
        plots_out['1'],
        plots_out['2'],
        plots_out['3'],
    ])
db_url = 'https://inf1007.firebaseio.com/.json?'
db_url = 'https://teststudentgradesdb.firebaseio.com/.json?'
controller.wg['url'].value = db_url
controller.apply_url()

In [ ]:
import webbrowser

_ = webbrowser.open_new('figures.html')

In [ ]:
from submissions_viewer.files.files_widgets import FilesWidgets

files_wg = FilesWidgets(out['submissions_viewer'], controller.db.filter_db)
files_wg.wg['organization'].value = 'TestOrgJustAymeric'